In [1]:
import trainer
from utils import get_batch_classifier_and_tagger, slanted_triangular_learning_rate
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
%matplotlib inline

In [2]:
with open('109_Bank/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('109_Bank/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)

In [3]:
VERSION = '109_Bank'
params=json.load(open('{}/trainer_params.json'.format(VERSION)))
params['optimizer']={'name': 'adamw', 'params': {'weight_decay': 1.2e-06}}
params['wdecay']=0.0
my_trainer = trainer.Trainer(**params, fine_tune=True)
my_trainer.logger.info('Trainer params {}'.format(params))
tf.reset_default_graph()

Trainer params {'log_path': '109_Bank/logs', 'clip_norm': 0.3, 'alpha': 1e-06, 'use_ema': True, 'negative_samples': 16384, 'train_summary_dir': '109_Bank/train_summary', 'model_configs': {'char_vocab_size': 366, 'skip_connection': True, 'projection_dims': 512, 'char_cnn_options': {'layers': [[1, 16], [2, 16], [3, 32], [4, 64], [5, 128], [6, 256], [7, 512]], 'n_highways': 2}, 'vocab_size': 68716, 'char_vec_size': 16, 'drop_e': 0.4, 'rnn_layers': [{'drop_i': 0.2, 'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}, {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}, {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}]}, 'test_summary_dir': '109_Bank/test_summary', 'beta': 1e-06, 'checkpoint_dir': '109_Bank/checkpoints', 'optimizer': {'name': 'adamw', 'params': {'weight_decay': 1.2e-06}}, 'save_freq': 1000, 'wdecay': 0.0, 'bptt': 128}


In [4]:
with open('Bank/class2idx.json', 'r') as inp:
    class2idx = json.load(inp)
with open('Bank/tag2idx.json', 'r') as inp:
    tag2idx = json.load(inp)

In [5]:
classifier_configs = {
    'layers': [
        {'batch_norm': False, 'drop_out': 0.5},
#         {'units': 50, 'batch_norm': False, 'drop_out': 0.1, 'activation': 'relu'}
    ], 'n_classes': len(class2idx)
}
tagger_configs = {
    'layers': [
        {'batch_norm': False, 'drop_out': 0.5}
    ], 'n_classes': len(tag2idx)
}
my_trainer.build_classifier_and_sequence_tagger(classifier_configs, tagger_configs, 
                                                folder_name='chatbot_train')

/root/anaconda2/envs/tfp/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from 109_Bank/checkpoints/chatbot_train/model.cpkt-86


Restoring parameters from 109_Bank/checkpoints/chatbot_train/model.cpkt-86


In [6]:
my_trainer.restore_language_model('109_Bank/checkpoints/test_bank/model-full.cpkt-898')

INFO:tensorflow:Restoring parameters from 109_Bank/checkpoints/test_bank/model-full.cpkt-898


Restoring parameters from 109_Bank/checkpoints/test_bank/model-full.cpkt-898


In [7]:
train_chars = np.load('Bank/chars.npy')
train_labels = np.load('Bank/labels.npy')
train_tags = np.load('Bank/tag_seqs.npy')

In [8]:
test_chars = train_chars
test_labels = train_labels
test_tags = train_tags

In [9]:
TRAIN_FOLDER = 'chatbot_train'
TEST_FOLDER = 'chatbot_test'
RATIO = 1/2.6
RATIO1 = RATIO/(1+RATIO)
BATCH_SIZE = 17
BPTT = 128

In [10]:
NUM_EPOCH = 15
TOTAL_ITER = NUM_EPOCH * len(train_labels)/BATCH_SIZE
LR_MAX = 3e-4
LR_RATIO = 32
lr = slanted_triangular_learning_rate(TOTAL_ITER, 1/NUM_EPOCH, LR_MAX, LR_RATIO)
for _ in range(NUM_EPOCH):
    my_trainer.train_step_classifier_tagger(train_chars, train_labels, train_tags, BATCH_SIZE, lr, BPTT, [0, 200], fine_tune_rate=[RATIO, RATIO1, 0, 0], folder_name=TRAIN_FOLDER)

Step   87: loss: 72.24789, class acc: 0.23529, tag acc: 0.00595, bptt: 129, time 02.85
Step   88: loss: 69.67773, class acc: 0.17647, tag acc: 0.00000, bptt: 132, time 03.02
Step   89: loss: 69.90887, class acc: 0.23529, tag acc: 0.01143, bptt: 127, time 03.16
Step   90: loss: 68.90942, class acc: 0.05882, tag acc: 0.00000, bptt: 128, time 03.28
Step   91: loss: 64.54464, class acc: 0.17647, tag acc: 0.00588, bptt: 137, time 03.41
Step   92: loss: 61.89086, class acc: 0.05882, tag acc: 0.01143, bptt: 129, time 03.51
Step   93: loss: 53.70599, class acc: 0.23529, tag acc: 0.02326, bptt: 126, time 03.64
Step   94: loss: 56.81437, class acc: 0.05882, tag acc: 0.01183, bptt: 129, time 03.76
Step   95: loss: 54.33466, class acc: 0.17647, tag acc: 0.01786, bptt: 125, time 03.86
Step   96: loss: 52.41733, class acc: 0.23529, tag acc: 0.05093, bptt: 134, time 04.18
Step   97: loss: 39.42014, class acc: 0.17647, tag acc: 0.03933, bptt: 123, time 04.33
Step   98: loss: 34.69266, class acc: 0.117

StopIteration: 

In [ ]:
from sklearn.metrics import classification_report

In [18]:
my_trainer.eval_step_classifier_tagger(test_chars, test_labels, test_tags, 10, 20, [0, 50, 200], TEST_FOLDER)

Evaluate total loss: 0.00013, total class acc: 1.00000, total tag acc: 1.00000, time 01.84
Evaluate total loss: 0.00210, total class acc: 1.00000, total tag acc: 1.00000, time 01.89
Evaluate total loss: 0.00207, total class acc: 1.00000, total tag acc: 1.00000, time 01.92
Evaluate total loss: 0.00160, total class acc: 1.00000, total tag acc: 1.00000, time 01.97
Evaluate total loss: 0.00128, total class acc: 1.00000, total tag acc: 1.00000, time 02.02
Evaluate total loss: 0.00112, total class acc: 1.00000, total tag acc: 1.00000, time 02.06
Evaluate total loss: 0.00097, total class acc: 1.00000, total tag acc: 1.00000, time 02.09
Evaluate total loss: 0.00085, total class acc: 1.00000, total tag acc: 1.00000, time 02.11
Evaluate total loss: 0.00084, total class acc: 1.00000, total tag acc: 1.00000, time 02.15
Evaluate total loss: 0.00076, total class acc: 1.00000, total tag acc: 1.00000, time 02.19
Evaluate total loss: 0.00111, total class acc: 1.00000, total tag acc: 1.00000, time 02.24